# Sesión 9

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

Descargamos los datos de https://archive.ics.uci.edu/ml/datasets/Glass+Identification

### Preparación de datos

In [ ]:
glass_data = pd.read_csv("./Data2/glass.csv")
glass_data.Type= glass_data.Type.str.replace('\'','')
glass_data.head()

In [ ]:
glass_data.loc[glass_data.Type == 'build wind float','Type']=1
glass_data.loc[glass_data.Type == 'build wind non-float','Type']=2
glass_data.loc[glass_data.Type == 'vehic wind float','Type']=3
glass_data.loc[glass_data.Type == 'containers','Type']=5
glass_data.loc[glass_data.Type == 'tableware','Type']=6
glass_data.loc[glass_data.Type == 'headlamps','Type']=7
glass_data['Type'] = pd.to_numeric(glass_data['Type'])

In [ ]:
#glass_data.hist("Type", grid="False")
#glass_data.hist("RI", grid="False")
#glass_data.hist("Na", grid="False")
glass_data.hist("K", grid="False")
#glass_data.hist("Ca", grid="False")

In [ ]:
print(len(glass_data))
glass_data = glass_data.drop(glass_data[glass_data['K']>=2].index)
print(len(glass_data))
glass_data.hist("K", grid="False")

In [ ]:
#glass_data.hist("RI", by="Type", grid="False", layout=[1,6],figsize = [10,3])
glass_data.hist("Type", grid="False",figsize = [5,3])

In [ ]:
print(len(glass_data))
new_df = glass_data.drop_duplicates()
print(len(new_df))
new_df = new_df.dropna()
print(len(new_df))

### Entrenamiento

In [ ]:
y_data = glass_data['Type']
x_data = glass_data.drop('Type', axis = 1)
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.2)

model = svm.SVC()
model.fit(x_training_data, y_training_data)

predictions = model.predict(x_test_data)
report = classification_report(y_test_data, predictions)
print(report)

Para más detalle revisar: https://scikit-learn.org/stable/modules/model_evaluation.html#classification-report

Para la interpretación revisar: https://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/

### Hiperparametro

In [ ]:
model1 = svm.SVC()
param_grid = {'C': [0.001, 0.01, 0.05, 0.1],
              'gamma': [0.001, 0.01, 0.05, 0.1],
              'kernel': ['rbf','poly'],
              'probability':[True] }

gs = GridSearchCV(model1, param_grid=param_grid, scoring='accuracy', refit='accuracy', cv=5, return_train_score=True)
gs.fit(x_training_data, y_training_data)
modelo = gs.best_estimator_
predictions = modelo.predict(x_test_data)
report = classification_report(y_test_data, predictions)
print(gs.best_estimator_)
print(report)

Para más detalle sobre param_grid revisar: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html

https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

Explicación del kernel: https://towardsdatascience.com/radial-basis-function-rbf-kernel-the-go-to-kernel-acf0d22c798a

https://medium.com/@myselfaman12345/c-and-gamma-in-svm-e6cee48626be

Referencias:

- chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/http://www.davidpublisher.com/Public/uploads/Contribute/56f9de9b67f83.pdf